## Testing of model

In [5]:
import google.generativeai as genai

genai.configure(api_key="")

model = genai.GenerativeModel("models/gemini-2.5-flash")
response = model.generate_content("You are a native of Western Odisha. Translate How are you? to Sambalpuri and also pure Odia. Give only translated text as output. Remember it should be in Odia Script")
print(response.text)


**Sambalpuri:**
କେନ୍ତା ଅଛନ୍?

**Pure Odia:**
କେମିତି ଅଛନ୍ତି?


## Listing available Models

In [3]:
import google.generativeai as genai


models = genai.list_models()

for model in models:
    print(model.name, "→", model.supported_generation_methods)


models/embedding-gecko-001 → ['embedText', 'countTextTokens']
models/gemini-2.5-flash → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp → ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-001 → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp-image-generation → ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash-lite-001 → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-lite → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-lite-preview-02-05 → ['generateContent', 'countTokens', '

## Choosing Gemini-3 Flash Preview for task 

Note - All API Keys are replaced by variables a,b,c,d,e

In [ ]:
import pandas as pd
import time
import random
import json
import re
from google import genai
from google.genai import types
from tenacity import retry, stop_after_attempt, wait_random_exponential

# 1. Initialize the Client
# IMPORTANT: Generate a NEW key in AI Studio. Your previous key was visible in the prompt.
client = genai.Client(api_key=a)

# 2. Define Research Instructions
instructions = (
    "You are an expert linguist specializing in the dialects of Odisha, India. "
    "Translate the English text into: 1) Standard Odia and 2) Sambalpuri dialect. "
    "Use Odia script. Format your output strictly as a JSON object: "
    "{'standard_odia': '...', 'sambalpuri': '...'}. Do not provide English explanations."
)

# 3. Define the Robust Inference Function
@retry(wait=wait_random_exponential(min=2, max=60), stop=stop_after_attempt(5))
def get_prediction(text):
    """Calls Gemini 2.5 Flash and manually parses the JSON text."""
    response = client.models.generate_content(
        model='gemini-2.5-flash', # Switched to 2.5 Flash for available quota
        contents=text,
        config=types.GenerateContentConfig(
            system_instruction=instructions,
            response_mime_type="application/json"
        )
    )
    
    if not response.text:
        return None

    try:
        # Clean potential Markdown artifacts and parse manually
        json_str = re.sub(r'```json\s*|```', '', response.text).strip()
        return json.loads(json_str)
    except json.JSONDecodeError:
        return None

# 4. Load dataset and run inference
df = pd.read_excel(r"\Sambalpuri_Sieve_SLM_Evaluation\Data\sambalpuri_std_odia_parallel_corpus.xlsx")
results = []

print("🚀 Starting Reliable Gemini 2.5 Flash Batch Inference...")

for index, row in df.iterrows():
    english_text = str(row['English_Source']).strip()
    
    if not english_text or english_text == "nan":
        continue
        
    try:
        data = get_prediction(english_text)
        
        if data and isinstance(data, dict):
            results.append({
                "AI_Odia": data.get('standard_odia', "PARSE_ERR"),
                "AI_SBP": data.get('sambalpuri', "PARSE_ERR")
            })
            print(f"✅ {index+1}/100 processed")
        else:
            results.append({"AI_Odia": "PARSE_ERR", "AI_SBP": "PARSE_ERR"})
            
    except Exception as e:
        print(f"❌ Error at {index+1}: {e}")
        results.append({"AI_Odia": "ERROR", "AI_SBP": "ERROR"})
    
    # CRITICAL: Sleep 15s to stay under the 5 RPM Free Tier limit
    time.sleep(15) 

# 5. Save results
df_results = pd.concat([df, pd.DataFrame(results)], axis=1)
df_results.to_excel(r"X:\Siddhanta\IIIT NR\6th Semester\Project_Sambalpuri\github\Sambalpuri_Sieve_SLM_Evaluation\Results\gemini_flash_outputs.xlsx", index=False)
print("🎯 Done! File saved.")

🚀 Starting Reliable Gemini 2.5 Flash Batch Inference...
✅ 1/100 processed
✅ 2/100 processed
✅ 3/100 processed
✅ 4/100 processed
✅ 5/100 processed
✅ 6/100 processed
✅ 7/100 processed
✅ 8/100 processed
✅ 9/100 processed
✅ 10/100 processed
✅ 11/100 processed
✅ 12/100 processed
✅ 13/100 processed
✅ 14/100 processed
✅ 15/100 processed
✅ 16/100 processed
✅ 17/100 processed
✅ 18/100 processed
✅ 19/100 processed
✅ 20/100 processed
✅ 21/100 processed
❌ Error at 22: RetryError[<Future at 0x24da7a5c250 state=finished raised ClientError>]
❌ Error at 23: RetryError[<Future at 0x24da7a5e1d0 state=finished raised ClientError>]
❌ Error at 24: RetryError[<Future at 0x24da7a5f100 state=finished raised ClientError>]
❌ Error at 25: RetryError[<Future at 0x24da7a79690 state=finished raised ClientError>]
❌ Error at 26: RetryError[<Future at 0x24da7a123b0 state=finished raised ClientError>]


In [6]:
import pandas as pd
import time
import os
import json
import re
import csv
from google import genai
from google.genai import types

# 1. Setup
client = genai.Client(api_key=a)
checkpoint_file = r"X:\Siddhanta\IIIT NR\6th Semester\Project_Sambalpuri\github\Sambalpuri_Sieve_SLM_Evaluation\gemini_flash_checkpoints.csv"
instructions = (
    "You are an expert linguist specializing in the dialects of Odisha, India. "
    "Translate the English text into: 1) Standard Odia and 2) Sambalpuri dialect. "
    "Use Odia script. Format your output strictly as a JSON object: "
    "{'standard_odia': '...', 'sambalpuri': '...'}. Do not provide English explanations."
)


# 2. Load Dataset
df = pd.read_excel(r"X:\Siddhanta\IIIT NR\6th Semester\Project_Sambalpuri\github\Sambalpuri_Sieve_SLM_Evaluation\Data\sambalpuri_std_odia_parallel_corpus.xlsx")

# 3. Resume Logic: Check which rows are already done
processed_indices = []
if os.path.exists(checkpoint_file):
    checkpoint_df = pd.read_csv(checkpoint_file)
    processed_indices = checkpoint_df['original_index'].tolist()
    print(f"🔄 Resuming... {len(processed_indices)} rows already finished.")

# 4. Open CSV in 'append' mode
with open(checkpoint_file, mode='a', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['original_index', 'English_Source', 'AI_Odia', 'AI_SBP'])
    
    # Write header only if file is new
    if not processed_indices:
        writer.writeheader()

    for index, row in df.iterrows():
        # Skip rows we've already done
        if index in processed_indices:
            continue
            
        english_text = str(row['English_Source']).strip()
        
        try:
            response = client.models.generate_content(
                model='gemini-3-flash-preview',
                contents=english_text,
                config=types.GenerateContentConfig(
                    system_instruction=instructions,
                    response_mime_type="application/json"
                )
            )
            
            # Clean and Parse
            json_str = re.sub(r'```json\s*|```', '', response.text).strip()
            data = json.loads(json_str)
            
            # Save IMMEDIATELY to disk
            writer.writerow({
                'original_index': index,
                'English_Source': english_text,
                'AI_Odia': data.get('standard_odia'),
                'AI_SBP': data.get('sambalpuri')
            })
            f.flush() # Forces Windows to write to the file now
            
            print(f"✅ {index+1} processed and SAVED.")
            
        except Exception as e:
            print(f"❌ Stopped at {index+1}: {e}")
            print("Daily quota likely exhausted. Run the script again tomorrow to resume.")
            break 
        
        time.sleep(15) # Stay under 5 RPM

print("🎯 Session complete.")

✅ 1 processed and SAVED.
✅ 2 processed and SAVED.
✅ 3 processed and SAVED.
✅ 4 processed and SAVED.
✅ 5 processed and SAVED.
✅ 6 processed and SAVED.
✅ 7 processed and SAVED.
✅ 8 processed and SAVED.
✅ 9 processed and SAVED.
✅ 10 processed and SAVED.
✅ 11 processed and SAVED.
✅ 12 processed and SAVED.
✅ 13 processed and SAVED.
✅ 14 processed and SAVED.
✅ 15 processed and SAVED.
✅ 16 processed and SAVED.
✅ 17 processed and SAVED.
✅ 18 processed and SAVED.
✅ 19 processed and SAVED.
✅ 20 processed and SAVED.
❌ Stopped at 21: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-3-flash\nPlease retry in 57.025012352s.', 'st

In [20]:
import pandas as pd
import time
import os
import json
import re
import csv
from google import genai
from google.genai import types

# 1. Setup
client = genai.Client(api_key=b)
checkpoint_file = r"X:\Siddhanta\IIIT NR\6th Semester\Project_Sambalpuri\github\Sambalpuri_Sieve_SLM_Evaluation\gemini_flash_checkpoints.csv"
instructions = (
    "You are an expert linguist specializing in the dialects of Odisha, India. "
    "Translate the English text into: 1) Standard Odia and 2) Sambalpuri dialect. "
    "Use Odia script. Format your output strictly as a JSON object: "
    "{'standard_odia': '...', 'sambalpuri': '...'}. Do not provide English explanations."
)


# 2. Load Dataset
df = pd.read_excel(r"X:\Siddhanta\IIIT NR\6th Semester\Project_Sambalpuri\github\Sambalpuri_Sieve_SLM_Evaluation\Data\sambalpuri_std_odia_parallel_corpus.xlsx")

# 3. Resume Logic: Check which rows are already done
processed_indices = []
if os.path.exists(checkpoint_file):
    checkpoint_df = pd.read_csv(checkpoint_file)
    processed_indices = checkpoint_df['original_index'].tolist()
    print(f"🔄 Resuming... {len(processed_indices)} rows already finished.")

# 4. Open CSV in 'append' mode
with open(checkpoint_file, mode='a', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['original_index', 'English_Source', 'AI_Odia', 'AI_SBP'])
    
    # Write header only if file is new
    if not processed_indices:
        writer.writeheader()

    for index, row in df.iterrows():
        # Skip rows we've already done
        if index in processed_indices:
            continue
            
        english_text = str(row['English_Source']).strip()
        
        try:
            response = client.models.generate_content(
                model='gemini-3-flash-preview',
                contents=english_text,
                config=types.GenerateContentConfig(
                    system_instruction=instructions,
                    response_mime_type="application/json"
                )
            )
            
            # Clean and Parse
            json_str = re.sub(r'```json\s*|```', '', response.text).strip()
            data = json.loads(json_str)
            
            # Save IMMEDIATELY to disk
            writer.writerow({
                'original_index': index,
                'English_Source': english_text,
                'AI_Odia': data.get('standard_odia'),
                'AI_SBP': data.get('sambalpuri')
            })
            f.flush() # Forces Windows to write to the file now
            
            print(f"✅ {index+1} processed and SAVED.")
            
        except Exception as e:
            print(f"❌ Stopped at {index+1}: {e}")
            print("Daily quota likely exhausted. Run the script again tomorrow to resume.")
            break 
        
        time.sleep(15) # Stay under 5 RPM

print("🎯 Session complete.")

🔄 Resuming... 20 rows already finished.
✅ 21 processed and SAVED.
✅ 22 processed and SAVED.
✅ 23 processed and SAVED.
✅ 24 processed and SAVED.
✅ 25 processed and SAVED.
✅ 26 processed and SAVED.
✅ 27 processed and SAVED.
✅ 28 processed and SAVED.
✅ 29 processed and SAVED.
✅ 30 processed and SAVED.
✅ 31 processed and SAVED.
✅ 32 processed and SAVED.
✅ 33 processed and SAVED.
✅ 34 processed and SAVED.
✅ 35 processed and SAVED.
✅ 36 processed and SAVED.
✅ 37 processed and SAVED.
✅ 38 processed and SAVED.
✅ 39 processed and SAVED.
✅ 40 processed and SAVED.
✅ 41 processed and SAVED.
❌ Stopped at 42: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_reque

In [22]:
import pandas as pd
import time
import os
import json
import re
import csv
from google import genai
from google.genai import types

# 1. Setup
client = genai.Client(api_key=c)
checkpoint_file = r"X:\Siddhanta\IIIT NR\6th Semester\Project_Sambalpuri\github\Sambalpuri_Sieve_SLM_Evaluation\gemini_flash_checkpoints.csv"
instructions = (
    "You are an expert linguist specializing in the dialects of Odisha, India. "
    "Translate the English text into: 1) Standard Odia and 2) Sambalpuri dialect. "
    "Use Odia script. Format your output strictly as a JSON object: "
    "{'standard_odia': '...', 'sambalpuri': '...'}. Do not provide English explanations."
)


# 2. Load Dataset
df = pd.read_excel(r"X:\Siddhanta\IIIT NR\6th Semester\Project_Sambalpuri\github\Sambalpuri_Sieve_SLM_Evaluation\Data\sambalpuri_std_odia_parallel_corpus.xlsx")

# 3. Resume Logic: Check which rows are already done
processed_indices = []
if os.path.exists(checkpoint_file):
    checkpoint_df = pd.read_csv(checkpoint_file)
    processed_indices = checkpoint_df['original_index'].tolist()
    print(f"🔄 Resuming... {len(processed_indices)} rows already finished.")

# 4. Open CSV in 'append' mode
with open(checkpoint_file, mode='a', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['original_index', 'English_Source', 'AI_Odia', 'AI_SBP'])
    
    # Write header only if file is new
    if not processed_indices:
        writer.writeheader()

    for index, row in df.iterrows():
        # Skip rows we've already done
        if index in processed_indices:
            continue
            
        english_text = str(row['English_Source']).strip()
        
        try:
            response = client.models.generate_content(
                model='gemini-3-flash-preview',
                contents=english_text,
                config=types.GenerateContentConfig(
                    system_instruction=instructions,
                    response_mime_type="application/json"
                )
            )
            
            # Clean and Parse
            json_str = re.sub(r'```json\s*|```', '', response.text).strip()
            data = json.loads(json_str)
            
            # Save IMMEDIATELY to disk
            writer.writerow({
                'original_index': index,
                'English_Source': english_text,
                'AI_Odia': data.get('standard_odia'),
                'AI_SBP': data.get('sambalpuri')
            })
            f.flush() # Forces Windows to write to the file now
            
            print(f"✅ {index+1} processed and SAVED.")
            
        except Exception as e:
            print(f"❌ Stopped at {index+1}: {e}")
            print("Daily quota likely exhausted. Run the script again tomorrow to resume.")
            break 
        
        time.sleep(15) # Stay under 5 RPM

print("🎯 Session complete.")

🔄 Resuming... 41 rows already finished.
✅ 42 processed and SAVED.
✅ 43 processed and SAVED.
✅ 44 processed and SAVED.
✅ 45 processed and SAVED.
✅ 46 processed and SAVED.
✅ 47 processed and SAVED.
✅ 48 processed and SAVED.
✅ 49 processed and SAVED.
✅ 50 processed and SAVED.
✅ 51 processed and SAVED.
✅ 52 processed and SAVED.
✅ 53 processed and SAVED.
✅ 54 processed and SAVED.
✅ 55 processed and SAVED.
✅ 56 processed and SAVED.
✅ 57 processed and SAVED.
✅ 58 processed and SAVED.
✅ 59 processed and SAVED.
✅ 60 processed and SAVED.
✅ 61 processed and SAVED.
✅ 62 processed and SAVED.
❌ Stopped at 63: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_reque

In [24]:
import pandas as pd
import time
import os
import json
import re
import csv
from google import genai
from google.genai import types

# 1. Setup
client = genai.Client(api_key=d)
checkpoint_file = r"X:\Siddhanta\IIIT NR\6th Semester\Project_Sambalpuri\github\Sambalpuri_Sieve_SLM_Evaluation\gemini_flash_checkpoints.csv"
instructions = (
    "You are an expert linguist specializing in the dialects of Odisha, India. "
    "Translate the English text into: 1) Standard Odia and 2) Sambalpuri dialect. "
    "Use Odia script. Format your output strictly as a JSON object: "
    "{'standard_odia': '...', 'sambalpuri': '...'}. Do not provide English explanations."
)


# 2. Load Dataset
df = pd.read_excel(r"X:\Siddhanta\IIIT NR\6th Semester\Project_Sambalpuri\github\Sambalpuri_Sieve_SLM_Evaluation\Data\sambalpuri_std_odia_parallel_corpus.xlsx")

# 3. Resume Logic: Check which rows are already done
processed_indices = []
if os.path.exists(checkpoint_file):
    checkpoint_df = pd.read_csv(checkpoint_file)
    processed_indices = checkpoint_df['original_index'].tolist()
    print(f"🔄 Resuming... {len(processed_indices)} rows already finished.")

# 4. Open CSV in 'append' mode
with open(checkpoint_file, mode='a', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['original_index', 'English_Source', 'AI_Odia', 'AI_SBP'])
    
    # Write header only if file is new
    if not processed_indices:
        writer.writeheader()

    for index, row in df.iterrows():
        # Skip rows we've already done
        if index in processed_indices:
            continue
            
        english_text = str(row['English_Source']).strip()
        
        try:
            response = client.models.generate_content(
                model='gemini-3-flash-preview',
                contents=english_text,
                config=types.GenerateContentConfig(
                    system_instruction=instructions,
                    response_mime_type="application/json"
                )
            )
            
            # Clean and Parse
            json_str = re.sub(r'```json\s*|```', '', response.text).strip()
            data = json.loads(json_str)
            
            # Save IMMEDIATELY to disk
            writer.writerow({
                'original_index': index,
                'English_Source': english_text,
                'AI_Odia': data.get('standard_odia'),
                'AI_SBP': data.get('sambalpuri')
            })
            f.flush() # Forces Windows to write to the file now
            
            print(f"✅ {index+1} processed and SAVED.")
            
        except Exception as e:
            print(f"❌ Stopped at {index+1}: {e}")
            print("Daily quota likely exhausted. Run the script again tomorrow to resume.")
            break 
        
        time.sleep(15) # Stay under 5 RPM

print("🎯 Session complete.")

🔄 Resuming... 62 rows already finished.
✅ 63 processed and SAVED.
✅ 64 processed and SAVED.
✅ 65 processed and SAVED.
✅ 66 processed and SAVED.
✅ 67 processed and SAVED.
✅ 68 processed and SAVED.
✅ 69 processed and SAVED.
✅ 70 processed and SAVED.
✅ 71 processed and SAVED.
✅ 72 processed and SAVED.
✅ 73 processed and SAVED.
✅ 74 processed and SAVED.
✅ 75 processed and SAVED.
✅ 76 processed and SAVED.
✅ 77 processed and SAVED.
✅ 78 processed and SAVED.
✅ 79 processed and SAVED.
✅ 80 processed and SAVED.
✅ 81 processed and SAVED.
✅ 82 processed and SAVED.
✅ 83 processed and SAVED.
❌ Stopped at 84: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_reque

In [26]:
import pandas as pd
import time
import os
import json
import re
import csv
from google import genai
from google.genai import types

# 1. Setup
client = genai.Client(api_key=e)
checkpoint_file = r"X:\Siddhanta\IIIT NR\6th Semester\Project_Sambalpuri\github\Sambalpuri_Sieve_SLM_Evaluation\gemini_flash_checkpoints.csv"
instructions = (
    "You are an expert linguist specializing in the dialects of Odisha, India. "
    "Translate the English text into: 1) Standard Odia and 2) Sambalpuri dialect. "
    "Use Odia script. Format your output strictly as a JSON object: "
    "{'standard_odia': '...', 'sambalpuri': '...'}. Do not provide English explanations."
)


# 2. Load Dataset
df = pd.read_excel(r"\Sambalpuri_Sieve_SLM_Evaluation\Data\sambalpuri_std_odia_parallel_corpus.xlsx")

# 3. Resume Logic: Check which rows are already done
processed_indices = []
if os.path.exists(checkpoint_file):
    checkpoint_df = pd.read_csv(checkpoint_file)
    processed_indices = checkpoint_df['original_index'].tolist()
    print(f"🔄 Resuming... {len(processed_indices)} rows already finished.")

# 4. Open CSV in 'append' mode
with open(checkpoint_file, mode='a', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['original_index', 'English_Source', 'AI_Odia', 'AI_SBP'])
    
    # Write header only if file is new
    if not processed_indices:
        writer.writeheader()

    for index, row in df.iterrows():
        # Skip rows we've already done
        if index in processed_indices:
            continue
            
        english_text = str(row['English_Source']).strip()
        
        try:
            response = client.models.generate_content(
                model='gemini-3-flash-preview',
                contents=english_text,
                config=types.GenerateContentConfig(
                    system_instruction=instructions,
                    response_mime_type="application/json"
                )
            )
            
            # Clean and Parse
            json_str = re.sub(r'```json\s*|```', '', response.text).strip()
            data = json.loads(json_str)
            
            # Save IMMEDIATELY to disk
            writer.writerow({
                'original_index': index,
                'English_Source': english_text,
                'AI_Odia': data.get('standard_odia'),
                'AI_SBP': data.get('sambalpuri')
            })
            f.flush() # Forces Windows to write to the file now
            
            print(f"✅ {index+1} processed and SAVED.")
            
        except Exception as e:
            print(f"❌ Stopped at {index+1}: {e}")
            print("Daily quota likely exhausted. Run the script again tomorrow to resume.")
            break 
        
        time.sleep(15) # Stay under 5 RPM

print("🎯 Session complete.")

🔄 Resuming... 83 rows already finished.
✅ 84 processed and SAVED.
✅ 85 processed and SAVED.
✅ 86 processed and SAVED.
✅ 87 processed and SAVED.
✅ 88 processed and SAVED.
✅ 89 processed and SAVED.
✅ 90 processed and SAVED.
✅ 91 processed and SAVED.
✅ 92 processed and SAVED.
✅ 93 processed and SAVED.
✅ 94 processed and SAVED.
✅ 95 processed and SAVED.
✅ 96 processed and SAVED.
✅ 97 processed and SAVED.
✅ 98 processed and SAVED.
✅ 99 processed and SAVED.
✅ 100 processed and SAVED.
🎯 Session complete.
